In [1]:
import os

Root = "/content/drive/MyDrive/NLP Projects/1. News Short"
os.chdir(Root)

In [11]:
pwd!

'/content/drive/MyDrive/NLP Projects/1. News Short'

In [2]:
pip install -r requirements.txt

  Running setup.py develop for news-short


In [ ]:
# !pip install transformers[torch]
# !pip install git+https://github.com/huggingface/accelerate

In [ ]:
!pip list

In [3]:
from src.load_and_save_data import DataLoader

In [4]:
data_loader = DataLoader()
data = data_loader.load_csv_data()

data.head()

,document,summary
0,"On a scorching hot day in June 2015, I gave wa...",– The activist who faces up to six months in j...
1,President Donald Trump has ramped up in recent...,– Over the weekend President Trump took to Twi...
2,"ABUJA, Nigeria (AP) — Nigeria's government and...",– A rep for Nigerian President Goodluck Jonath...
3,"Japan's Misao Okawa, 116, who is recognized as...",– Surrounded by flowers and family and wearing...
4,Italy Quake Destroys £200m Worth Of Cheese \n ...,– Northern Italy is still being hit by aftersh...


In [19]:
data.shape

(14992, 2)

In [ ]:
new_data = data[0:50]
new_data

In [6]:
data_loader.save_data(data=new_data)

In [7]:
from src.preprocessor import PreProcessor

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
process = PreProcessor()
# data = process.handle_data()
X, Y = process.separate_x_y(data=new_data)

In [9]:
X

0      Washington lurched toward another potential go...
1      Sign up for one of our email newsletters. \n \...
2      The X and Y chromosomes, with the smaller Y ch...
3      Image copyright NYPD Image caption John propos...
4      Republican presidential candidate and former M...
                             ...                        
195    KABUL/PESHAWAR, Pakistan (Reuters) - The Afgha...
196    Iraqi engineers involved in building the Mosul...
197    Labor Day weekend was going to be a turning po...
198    A veteran New York politician is demanding cit...
199    LEXINGTON, N.C. (AP) — An inmate who authoriti...
Name: document, Length: 200, dtype: object

In [10]:
Y

0      – House Republicans got their ducks in a row l...
1      – An "ambush" on two Pennsylvania police offic...
2      – Even if general appeals to one's health aren...
3      – British couple Daniella Anthony and John Dre...
4      – Mitt Romney tried yesterday to turn back the...
                             ...                        
195    – Senior Taliban members, after successfully e...
196    – Americans in Iraq should be poised to leave ...
197    – The injustice of it all is staggering: A Nor...
198    – Anthony Weiner has already lost wife Huma Ab...
199    – Another prison worker has aided another esca...
Name: summary, Length: 200, dtype: object

In [11]:
X = process.text_preprocessing(data=X)
Y = process.text_preprocessing(data=Y)

In [12]:
X_tok[0]

'washington lurch toward anoth potenti govern shutdown crisi friday hous approv vote gopauthor shortterm fund measur design keep govern run nov democrat senat immedi vow reject bill “ expect vote fairli quickli ” said senat major leader harri reid dnev friday morn aftermidnight roll call hous republican leader persuad conserv earli friday morn support stopgap measur nearli ident one reject hour earlier narrow margin republican support plan along six democrat democrat oppos join republican bill keep feder agenc fund nov pass staunch object hous democrat oppos provis would pair increas fund disast relief spend cut program make loan car compani encourag energi effici car product hous speaker john boehner ’ rohio victori like shortliv reid said late thursday measur could pass chamber vote expect sometim friday senat defeat would leav congress new standoff “ fail provid relief fellow american need struggl rebuild live wake flood wildfir hurrican reject senat ” reid said hous bill without re

In [8]:
df = new_data

In [9]:
from sklearn.model_selection import train_test_split

# Split the dataset into train, validation, and test sets
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.3, random_state=42)


train_X = train_df["document"].values
train_Y = train_df["summary"].values

val_X = val_df["document"].values
val_Y = val_df["summary"].values

test_X = test_df["document"].values
test_Y = test_df["summary"].values

In [ ]:
import torch
from transformers import T5Tokenizer

# Initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")


# Tokenize and encode the text and summary for training set
train_encodings = tokenizer(train_X.tolist(), text_pair=train_Y.tolist(), truncation=True, padding='max_length', max_length=512)
train_input_ids = torch.tensor(train_encodings["input_ids"], dtype=torch.long)
train_attention_mask = torch.tensor(train_encodings["attention_mask"], dtype=torch.long)
train_labels = torch.tensor(train_encodings["input_ids"], dtype=torch.long)  # Use input_ids as labels for T5

# Tokenize and encode the text and summary for validation set
valid_encodings = tokenizer(val_X.tolist(), text_pair=val_Y.tolist(), truncation=True, padding='max_length', max_length=512)
valid_input_ids = torch.tensor(valid_encodings["input_ids"], dtype=torch.long)
valid_attention_mask = torch.tensor(valid_encodings["attention_mask"], dtype=torch.long)
valid_labels = torch.tensor(valid_encodings["input_ids"], dtype=torch.long)  # Use input_ids as labels for T5

# Tokenize and encode the text and summary for test set
test_encodings = tokenizer(test_X.tolist(), text_pair=test_Y.tolist(), truncation=True, padding='max_length', max_length=512)
test_input_ids = torch.tensor(test_encodings["input_ids"], dtype=torch.long)
test_attention_mask = torch.tensor(test_encodings["attention_mask"], dtype=torch.long)
test_labels = torch.tensor(test_encodings["input_ids"], dtype=torch.long)  # Use input_ids as labels for T5



In [12]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5Config
from transformers import Trainer, TrainingArguments

# Load the "t5-small" model
model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Define the batch size and the maximum sequence length
batch_size = 8
max_seq_length = 512

# Create a PyTorch DataLoader for training, validation, and test datasets
train_dataset = torch.utils.data.TensorDataset(train_input_ids, train_attention_mask, train_labels)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset = torch.utils.data.TensorDataset(valid_input_ids, valid_attention_mask, valid_labels)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size)

test_dataset = torch.utils.data.TensorDataset(test_input_ids, test_attention_mask, test_labels)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)




In [34]:
test_dataset

In [15]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Step 3: Tokenization and Creating PyTorch Tensors
# (Code from the previous step)

# Step 4: Fine-tuning the "t5-small" model
model = T5ForConditionalGeneration.from_pretrained("t5-small")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def data_collator(features):
    input_ids = torch.stack([feature[0] for feature in features])
    attention_mask = torch.stack([feature[1] for feature in features])
    labels = torch.stack([feature[2] for feature in features])
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

training_args = TrainingArguments(
    output_dir='modellll',
    num_train_epochs=3, warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy='steps', eval_steps=500,
    save_steps=1e6,
    gradient_accumulation_steps=16
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator  # Use our custom data collator
)

trainer.train()

# Note: To perform inference (text summarization) using the fine-tuned model, you can load the model using T5ForConditionalGeneration.from_pretrained("./fine_tuned_model") and use the model.generate() method.


Step,Training Loss,Validation Loss


TrainOutput(global_step=3, training_loss=10.44216791788737, metrics={'train_runtime': 191.1703, 'train_samples_per_second': 0.377, 'train_steps_per_second': 0.016, 'total_flos': 6496406470656.0, 'train_loss': 10.44216791788737, 'epoch': 2.0})

In [16]:
# Step 6: Evaluating the model
results = trainer.evaluate()

# Step 7: Print evaluation results
print("Evaluation Results:")
for key, value in results.items():
    print(f"{key}: {value}")




Evaluation Results:
eval_loss: 10.257271766662598
eval_runtime: 14.0263
eval_samples_per_second: 0.784
eval_steps_per_second: 0.784
epoch: 2.0


In [17]:
model.save_pretrained("my-t5-small")

In [19]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("my-t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")


In [20]:
from transformers import pipeline

# Prepare the input text for summarization
input_text = "The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."

# Create a summarization pipeline
summarizer = pipeline(model=model, tokenizer=tokenizer, task="summarization")

# Generate the summarization using the pipeline
summary = summarizer(input_text, max_length=150, num_beams=2, early_stopping=True)

# Print the generated summary
print("Generated Summary: \n\n", summary[0]["summary_text"])


Generated Summary: 

 the Eiffel Tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building . it is the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in new york City was finished in 1930 .


In [ ]:
"""
  the Eiffel Tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building .
  it is the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in new york City was finished in 1930
"""

In [26]:


actual_input = df['document'][0]
actual_output = df['summary'][0]

actual_output

'– The activist who faces up to six months in jail and a $5,000 fine for giving water to thirsty pigs en route to a slaughterhouse is speaking out. "There’s little doubt in my mind that if those were dogs in distress in that truck instead of pigs, my actions would be applauded and it would be the driver facing charges instead," Anita Krajnc writes in the Guardian, describing pigs that were crowded in a transport truck in extreme heat, covered in their own feces, some of them foaming at the mouth or breathing as quickly as 180 breaths a minute. Pigs are actually a lot like dogs, with "a strong sense of self and intelligence," and "this double standard should have everyone questioning the ethics of the meat, dairy, and egg industry; our legal system; and our food choice." Indeed, humans themselves are animals as well, she points out, and we\'re like animals in many ways: We all suffer and feel fear and pain, and we all feel thirsty when we\'re hot. It\'s time our laws were shifted, she w

In [27]:
pred_out = summary = summarizer(actual_input, max_length=300)
pred_out = pred_out[0]["summary_text"]
pred_out

"anita Krajnc, 49, is on trial for giving water to thirsty pigs on a hot day . the trial resumes in a courtroom on monday . she says pig farmers should be treated as individuals with rights, not as property . a pig's father says he's not a vegan, but a vegetarian ."

In [30]:
import pandas as pd
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)
scores = scorer.score(pred_out, actual_output)

pd.DataFrame(scores, index=['precision', 'recall', 'fmeasure'])

,rouge1,rouge2,rougeL,rougeLsum
precision,0.128514,0.040323,0.080321,0.080321
recall,0.627451,0.200000,0.392157,0.392157
fmeasure,0.213333,0.067114,0.133333,0.133333


In [ ]:
train_dataset[0]